# Optic Chiasm - Tutorial

The [optic chiasm](https://en.wikipedia.org/wiki/Optic_chiasm) is a structure in the brain, where the two nerve fiber bundles from the eyes cross each other.
However not all of the fiber cross the structure.
Depending on the species, a fraction of the nerve fibers stay on the same hemisphere.

### Note

> The here choosen fibers radii and their layer parameters are not realistic. They are just for a fast computation.


## General imports

In [ ]:
import fastpli.model.sandbox
import fastpli.model.solver
import fastpli.simulation
import fastpli.analysis
import fastpli.tools
import fastpli.io

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import multiprocessing as mp
import numpy as np
import scipy.interpolate as si

pool = mp.Pool(2)
np.random.seed(42)

Here we just do a simple representation to get an idea of how we can build such a structure.

We start of defining the main fiber bundle geometries, which we will later fill with individual fibers.

We need two crossing trajectories `fb_0` and `fb_1` as well as two kissing trajectories `fb_left` and `fb_right`.
To help for a smoother curve, we use the scipy interpolation function.

In [ ]:

# two crossing main bundle
fb_0 = np.array([[-2000, -2000, 0], [-1000, -1000, 0], [0,0,0], [1000, 1000,0], [2000, 2000, 0]])
fb_1 = np.array([[-2000, 2000, 0], [-1000, 1000, 0], [0,0,0], [1000, -1000,0], [2000, -2000, 0]])

# left and right non crossing bundle
fb_left = np.array([[-2000,-2000,0], [-1050,-1000,0], [-300,0,0], [-1050,1000,0], [-2000,2000,0]])
fb_right = fb_left.copy()
fb_right[:,0] *= -1

# interpolate left and right bundle for smoother
N = 20
t = np.linspace(0,1,fb_left.shape[0])
t_intp = np.linspace(0,1,N)
fb_left = np.array([si.interp1d(t, fb_left[:,0], 'quadratic')(t_intp), si.interp1d(t, fb_left[:,1], 'quadratic')(t_intp), si.interp1d(t, fb_left[:,2], 'quadratic')(t_intp)]).T
fb_right = np.array([si.interp1d(t, fb_right[:,0], 'quadratic')(t_intp), si.interp1d(t, fb_right[:,1], 'quadratic')(t_intp), si.interp1d(t, fb_right[:,2], 'quadratic')(t_intp)]).T

fig, ax = plt.subplots(1,1)
plt.plot(fb_0[:,0], fb_0[:,1])
plt.plot(fb_1[:,0], fb_1[:,1])
plt.plot(fb_left[:,0], fb_left[:,1])
plt.plot(fb_right[:,0], fb_right[:,1])
ax.axis('equal')
plt.show()


Now we can fill the bundles with fibers.
To account for additional varirity we use a random sampling of radii.
The `sandbox.build.bundle` methods helps us to generate the filled fiber bundles `ffb_*`

In [ ]:
population = fastpli.model.sandbox.seeds.triangular_circle(800, 200)
fiber_radii = np.random.uniform(50.0, 100.0, population.shape[0])
ffb_0 = fastpli.model.sandbox.build.bundle(fb_0, population, fiber_radii)
ffb_1 = fastpli.model.sandbox.build.bundle(fb_1, population, fiber_radii)

population = fastpli.model.sandbox.seeds.triangular_circle(400, 200)
fiber_radii = np.random.uniform(50.0, 100.0, population.shape[0])
ffb_left = fastpli.model.sandbox.build.bundle(fb_left, population, fiber_radii)
ffb_right = fastpli.model.sandbox.build.bundle(fb_right, population, fiber_radii)

Now we have our initial structure.
However the individual fiber most certanly collide with each other.
Therefore we use the `Solver` to help us.

### Note

> The `Solver` class provides a `Solver.draw_scene()` method.
> However you should not use this in a jupyter notebook, since its current design does not allow a second initialization of the underlying _OpenGL_ structure.
> See `examples/optic_chiasm.py` for this.
> To have a animated view, we use IPython's display function

> Also be carefull with the number of fibers and fiber segments (i. e. obj_mean_length).
> Depending on the configurations, the computational complexity can increase by `O(n_segments^2)`

In [ ]:
from IPython import display
fig, ax = plt.subplots(1,1)

solver = fastpli.model.solver.Solver()
solver.fiber_bundles = [ffb_0, ffb_1, ffb_left, ffb_right]
solver.obj_min_radius = 150
solver.obj_mean_length = 150
solver.omp_num_threads = 2

for i in range(1000):
    solved = solver.step()
    if i % 42 == 0:
        for fb in solver.fiber_bundles:
            for f in fb:
                # show only the middle section of fibers
                if np.any(f[:,2] > 50) or np.any(f[:,2] < -50):
                    plt.plot(f[:,0], f[:,1])
        ax.axis('equal')
        ax.set_title(f'step: {i}')
        display.clear_output(wait=True)
        display.display(fig)
    if solved:
        break

Now that we have our model we want to simulate it inside a virtual 3D-PLI microscope.
We cut out a brain section of 4000x4000x100 µm^3.

### Note

> The simulation takes quite a lot of memory. It's size increases with `O(n_x*n_y*n_z)`.
> The `Simpli.memory_usage()` method gives you a hint how much memory is needed.

In [ ]:
simpli = fastpli.simulation.Simpli()
simpli.omp_num_threads = 2
simpli.voxel_size = 5  # in micro meter
simpli.set_voi([-2000, -2000, -50], [2000, 2000, 50])  # in micro meter
simpli.fiber_bundles = solver.fiber_bundles
simpli.fiber_bundles.layers = [[(1.0, -0.0001, 5, 'p')]] * len(
        simpli.fiber_bundles)
print('Memory:', str(round(simpli.memory_usage('MB'), 2)) + ' MB')        

Next we generate the discreticed volume

In [ ]:
tissue, optical_axis, tissue_properties = simpli.generate_tissue()

and after defining the microscopic parameters

In [ ]:
simpli.filter_rotations = np.deg2rad([0, 30, 60, 90, 120, 150])
simpli.light_intensity = 26000  # a.u.
simpli.wavelength = 525  # in nm
simpli.pixel_size = 20  # in micro meter
simpli.optical_sigma = 0.71  # in pixel size
simpli.noise_model = lambda x: np.random.negative_binomial(
    x / (3 - 1), 1 / 3)
simpli.tilts = np.deg2rad([(0, 0), (5.5, 0), (5.5, 90),
                            (5.5, 180), (5.5, 270)])

we can simulate and analyse our measurement

In [ ]:
tilting_stack = []
for theta, phi in simpli.tilts:
    print(f"Tilt: theta={np.rad2deg(theta):.1f}, phi={np.rad2deg(phi):.1f}")
    images = simpli.run_simulation(tissue, optical_axis, tissue_properties,
                                    theta, phi)

    _, images = simpli.apply_optic(images)
    tilting_stack.append(images)

transmittance, direction, retardation = simpli.apply_epa(tilting_stack[0])
rofl_direction, rofl_incl, rofl_t_rel, _ = simpli.apply_rofl(tilting_stack, mp_pool=pool)
fom = fastpli.analysis.images.fom_hsv_black(rofl_direction, rofl_incl)

### Note

All datasets containing spacial information (images, tissue, optical axis) are orded in x-y-z. Since Matplotlib and mostly other visualization libraries or image datasets expect a order of y-x, we have to swap the first two axis first. Additionally we also have to set the origin to 'lower'. Also it is recommendet to use the interpolation method 'nearest'.

In [ ]:
fig, axs = plt.subplots(1,3, sharey=True, figsize=(15,5))
axs[0].imshow(transmittance.T, origin='lower', interpolation='nearest')
axs[0].set_title('transmittance')
axs[1].imshow(direction.T, origin='lower', interpolation='nearest')
axs[1].set_title('direction')
axs[2].imshow(retardation.T, origin='lower', interpolation='nearest')
axs[2].set_title('retardation')

In [ ]:
fig, axs = plt.subplots(1,4, sharey=True, figsize=(20,5))
axs[0].imshow(rofl_direction.T, origin='lower', interpolation='nearest')
axs[0].set_title('direction')
axs[1].imshow(rofl_incl.T, origin='lower', interpolation='nearest')
axs[1].set_title('inclination')
axs[2].imshow(rofl_t_rel.T, origin='lower', interpolation='nearest')
axs[2].set_title('t_rel')
axs[3].imshow(np.swapaxes(fom, 0, 1), origin='lower', interpolation='nearest')
axs[3].set_title('fom')
plt.show()

fig, axs = plt.subplots(1,1, figsize=(1,1))
plt.imshow(np.swapaxes(fastpli.analysis.images.hsv_black_sphere(), 0, 1), origin='lower')
plt.axis('off')